## Pareto-Pruner Visualization
This notebook plots each evaluated flow using a color that indicates the number of samples evaluated. It visualizes nicely how the use of Pareto-Pruner leads to only a few evaluations far away from the Pareto-frontier.

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core import ultratb

ultratb.VerboseTB.tb_highlight = "bg:#3e0054"

In [ ]:
import matplotlib.pyplot as plt

# you need to restart the notebook for changes to take effect
USE_DARK_STYLE = False

if USE_DARK_STYLE:
    plt.style.use('dark_background')

In [ ]:
from pathlib import Path
import optuna
import pandas as pd

from syftr.configuration import cfg
from syftr.optuna_helper import get_completed_trials

STUDY_NAME = "rank0--rag-and-agents--financebench_hf"

USE_COSTS = True

OBJ1_NAME = cfg.plotting.target_accuracy_name
OBJ1_UNIT = cfg.plotting.target_accuracy_unit
if USE_COSTS:
    OBJ2_NAME = cfg.plotting.target_cost_name
    OBJ2_UNIT = cfg.plotting.target_cost_unit
else:
    OBJ2_NAME = cfg.plotting.target_latency_name
    OBJ2_UNIT = cfg.plotting.target_latency_unit

DATAROBOT_PURPLE = cfg.plotting.datarobot_purple
DATAROBOT_GREEN = cfg.plotting.datarobot_green
DATAROBOT_YELLOW = cfg.plotting.datarobot_yellow

COLOR_MARKER = DATAROBOT_YELLOW if USE_DARK_STYLE else "black"

# SUCCESS_RATE = 0.9
SUCCESS_RATE = None

STORAGE = cfg.postgres.get_optuna_storage()

RESULTS_DIR: Path = cfg.paths.results_dir
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR = RESULTS_DIR.resolve()

study = optuna.load_study(study_name=STUDY_NAME, storage=STORAGE)
df: pd.DataFrame = get_completed_trials(study, success_rate=SUCCESS_RATE)

cfg.paths.results_dir.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

print(f"The study '{STUDY_NAME}' has {len(df)} completed trials")

In [ ]:
def postprocess(trials_dataframe: pd.DataFrame) -> pd.DataFrame:
    df = trials_dataframe.copy()
    assert 0 == len(df[df["values_1"] == 0]), "Zero-Accuracy result detected"
    assert all(~df["user_attrs_flow"].isna()), "Missing flow"
    df = df.sort_values("user_attrs_metric_eval_end")
    return df.reset_index(drop=True)

In [ ]:
df = postprocess(df)

In [ ]:
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from paretoset import paretoset

x_min = df["values_1"].min()
x_max = df["values_1"].max()

pareto_mask_pruned = paretoset(
    df[["values_0", "values_1"]], sense=["max", "min"]
)
df_pareto_pruned = df[pareto_mask_pruned].copy()
df_pareto_pruned = df_pareto_pruned.sort_values("values_1")

def pareto_curve(x):
    return np.interp(x, df_pareto_pruned["values_1"].values, df_pareto_pruned["values_0"].values)

cmap = LinearSegmentedColormap.from_list(
    "custom_cmap", [DATAROBOT_GREEN, DATAROBOT_PURPLE]
)

norm = plt.Normalize(
    min(df["user_attrs_metric_num_total"]),
    max(df["user_attrs_metric_num_total"]),
)

scatter = plt.scatter(
    df["values_1"],
    100 * df["values_0"],
    s=10,
    c=df["user_attrs_metric_num_total"],
    cmap=cmap,
    norm=norm,
    # alpha=0.5,
)
cbar = plt.colorbar(scatter)
cbar.set_label("Number of Evaluations")

x_sample = np.linspace(
    start=df["values_1"].min(),
    stop=df["values_1"].max(),
    num=1000,
)
y_sample = pareto_curve(x_sample)
plt.plot(x_sample, 100 * y_sample, label="Pareto Frontier", color=DATAROBOT_PURPLE)
plt.legend()
plt.xlabel(OBJ2_UNIT)
plt.ylabel(OBJ1_UNIT)
plt.title(f"Reducing Number of Evaluations with ParetoPruner\n({STUDY_NAME})")

plt.tight_layout()
plt.savefig(cfg.paths.results_dir / f"{STUDY_NAME}-pruner.png")
plt.show()

In [ ]:
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

# df_trials = df_trials.iloc[:100]
fig, ax = plt.subplots(figsize=(8, 6))

cmap = LinearSegmentedColormap.from_list(
    "custom_cmap", [DATAROBOT_GREEN, DATAROBOT_PURPLE]
)
norm = Normalize(
    vmin=min(df["user_attrs_metric_num_total"]),
    vmax=max(df["user_attrs_metric_num_total"]),
)
colors = cmap(norm(df["user_attrs_metric_num_total"]))

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label("Number of Evaluations")

scat_data = ax.scatter(
    [],
    [],
    edgecolor="gray",
    linewidths=0.5,
    s=20,
)
(scat_pareto,) = ax.plot(
    [df["values_1"][0]],
    [df["values_0"][0]],
    marker="o",
    color=DATAROBOT_PURPLE,
    label="Pareto Frontier",
    markersize=5,
)
scat_pareto.set_data([], [])

ax.set_xlabel(OBJ2_UNIT)
ax.set_ylabel(OBJ1_UNIT)
ax.set_title(f"ParetoPruner (Zoom-in)")

text = ax.text(0.02, 0.95, "", transform=ax.transAxes)

ax.set_xlim(-0.005 * df["values_1"].max(), 0.5 * df["values_1"].max())
ax.set_ylim(0.9 * df["values_0"].min(), 1.1 * df["values_0"].max())

ax.legend()
plt.tight_layout()

rectangles = []
crosses = []
points = []
for _ in range(len(df)):
    rect = Rectangle((0, 0), 0, 0, color='none')
    rectangles.append(rect)
    ax.add_patch(rect)
    line_hor = Line2D([], [], color="black", linewidth=0.5)
    line_ver = Line2D([], [], color="black", linewidth=0.5)
    crosses.append((line_hor, line_ver))
    ax.add_line(line_hor)
    ax.add_line(line_ver)
    # (point,) = ax.plot([], [], "*", markersize=5, color=COLOR_MARKER)
    # points.append(point)


def update(frame):
    print(f"Processing frame: {frame}")
    text.set_text(f"Trial: {frame + 2}")

    df_tmp = df.iloc[:frame + 2]
    data = df_tmp.iloc[frame + 1]

    x = data["values_1"]
    y = data["values_0"]
    x_err = data["user_attrs_metric_obj2_confidence"]
    y_err = data["user_attrs_metric_obj1_confidence"]

    rectangle = rectangles[frame + 1]
    rectangle.set_xy((x - 0.5 * x_err, y - 0.5 * y_err))
    rectangle.set_width(x_err)
    rectangle.set_height(y_err)
    rectangle.set_color(colors[frame + 1])

    rectangle_before = rectangles[frame]
    rectangle_before.set_xy((0, 0))
    rectangle_before.set_width(0)
    rectangle_before.set_height(0)
    rectangle_before.set_color("none")

    center_x = x
    center_y = y
    line_hor, line_ver = crosses[frame + 1]
    line_hor.set_data([center_x - 0.5 * x_err, center_x + 0.5 * x_err], [center_y, center_y])
    line_ver.set_data([center_x, center_x], [center_y - 0.5 * y_err, center_y + 0.5 * y_err])

    line_hor_before, line_ver_before = crosses[frame]
    line_hor_before.set_data(
        [], []
    )
    line_ver_before.set_data(
        [], []
    )

    # point = points[frame + 1]
    # top_left_x = x - 0.5 * x_err
    # top_left_y = y + 0.5 * y_err
    # point.set_data([top_left_x], [top_left_y])

    # point_before = points[frame]
    # point_before.set_data([], [])

    scat_data.set_offsets(
        np.column_stack((df_tmp["values_1"], df_tmp["values_0"]))
    )
    scat_data.set_color(colors[: frame + 1])
    scat_data.set_linewidth(1)

    cbar.update_normal(sm)

    df_p = df_tmp[["values_0", "values_1"]].iloc[:frame + 1]
    mask = paretoset(df_p, sense=["max", "min"])
    pareto_indices = df_p[mask].index
    df_pareto: pd.DataFrame = df_p.loc[pareto_indices].copy()
    df_pareto = df_pareto.sort_values(by="values_0")
    scat_pareto.set_data(df_pareto["values_1"], df_pareto["values_0"])

    return (
        rectangle,
        rectangle_before,
        line_hor,
        line_ver,
        line_hor_before,
        line_ver_before,
        # point,
        # point_before,
        scat_data,
        scat_pareto,
        text,
    )

ani = animation.FuncAnimation(
    fig, update, frames=len(df) - 1, blit=True
)

ani.save(RESULTS_DIR / f"{STUDY_NAME}-pruner.mp4", writer="ffmpeg", fps=4)